In [16]:
print('Multi_Agent_Rag')

Multi_Agent_Rag


# PDF

In [17]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
os.environ["OPENAI_API_KEY"]=os.getenv("openai_api_key")
os.environ["LANGCHAIN_TRACING_V2"]="True"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("langsmith_api_key")

In [19]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("attention_is_all_you_need.pdf")
Attention=loader.load()

In [20]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=800,chunk_overlap=200)
Attention_split=text_splitter.split_documents(Attention)

In [21]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

In [22]:
Attention_db=Chroma.from_documents(Attention_split,OpenAIEmbeddings())

In [23]:
pdf_retriever=Attention_db.as_retriever()
pdf_retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000290E67C7620>, search_kwargs={})

In [24]:
from langchain.tools.retriever import create_retriever_tool
pdf_retriever_tool=create_retriever_tool(pdf_retriever,"Pdf_search","A tool to search for and extract information from PDF documents")

In [25]:
tools=[pdf_retriever_tool]
tools

[Tool(name='Pdf_search', description='A tool to search for and extract information from PDF documents', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000290E0FF2D40>, retriever=VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000290E67C7620>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x00000290E14189A0>, retriever=VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000290E67C7620>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_sepa

In [26]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4-turbo",temperature=0)

In [27]:
from langchain import hub
Prompt = hub.pull("hwchase17/openai-functions-agent")
Prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

## Agents

In [28]:
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,Prompt)

In [29]:
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [32]:
agent_executor.invoke({"input":"Who are authors of paper Attention is all you need"})



> Entering new AgentExecutor chain...

Invoking: `Pdf_search` with `{'query': 'authors of the paper Attention is all you need'}`


Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.eduŁukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best

[16] Łukasz Kaiser and Samy Bengio. Can active memory replace attention? In Advances in Neural
Information Processin

{'input': 'Who are authors of paper Attention is all you need',
 'output': 'The authors of the paper "Attention Is All You Need" are:\n\n- Ashish Vaswani\n- Noam Shazeer\n- Niki Parmar\n- Jakob Uszkoreit\n- Llion Jones\n- Aidan N. Gomez\n- Łukasz Kaiser\n- Illia Polosukhin'}

In [ ]:
agent_executor.invoke({"input":"What is query,key and value ?"})



> Entering new AgentExecutor chain...
In the context of data management and programming, the terms "query," "key," and "value" have specific meanings:

1. **Query:**
   - A query is a request for information or data from a database, system, or specific data structure. In databases, a query is often written in a query language, such as SQL (Structured Query Language), which allows the user to specify what data is needed and how it should be processed or filtered. Queries can also be used in other contexts, such as web search engines or APIs, where they represent a request for data or some operation.

2. **Key:**
   - A key is an identifier used in data structures, such as arrays, maps, or databases, to uniquely identify an element or a record. In databases, a key is used to find and retrieve data quickly. For example, in a relational database, a primary key is a specific choice of a minimal set of attributes (columns) that uniquely specify a tuple (row) in a table. In programming, key

{'input': 'What is query,key and value ?',
 'output': 'In the context of data management and programming, the terms "query," "key," and "value" have specific meanings:\n\n1. **Query:**\n   - A query is a request for information or data from a database, system, or specific data structure. In databases, a query is often written in a query language, such as SQL (Structured Query Language), which allows the user to specify what data is needed and how it should be processed or filtered. Queries can also be used in other contexts, such as web search engines or APIs, where they represent a request for data or some operation.\n\n2. **Key:**\n   - A key is an identifier used in data structures, such as arrays, maps, or databases, to uniquely identify an element or a record. In databases, a key is used to find and retrieve data quickly. For example, in a relational database, a primary key is a specific choice of a minimal set of attributes (columns) that uniquely specify a tuple (row) in a table

# Excel

In [6]:
import nest_asyncio
nest_asyncio.apply()

In [7]:
from llama_parse import LlamaParse
llama_key=os.getenv("llama_api_key")

In [8]:
parser=LlamaParse(api_key=llama_key,result_type="markdown")

In [9]:
documents=parser.load_data("pokemon_data.csv")

Started parsing the file under job_id f360318e-0e4f-4c76-b81f-4f715e89d101


In [10]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

In [11]:
gpt=OpenAI(model="gpt-4-turbo")
Settings.llm=gpt

In [12]:
from llama_index.core import VectorStoreIndex
index=VectorStoreIndex.from_documents(documents)

In [13]:
query_engine=index.as_query_engine()

In [14]:
response=query_engine.query("Difference between KyuremBlack Kyurem and KyuremWhite Kyurem ?")
str(response)

"The differences between Kyurem Black Kyurem and Kyurem White Kyurem are as follows:\n\n1. Attack and Special Attack stats:\n   - Kyurem Black Kyurem has a higher Attack stat of 170 compared to Kyurem White Kyurem's Attack stat of 120.\n   - Kyurem White Kyurem has a higher Special Attack stat of 170 compared to Kyurem Black Kyurem's Special Attack stat of 120.\n\n2. Defense and Special Defense stats:\n   - Both forms have the same Defense stat of 100.\n   - Kyurem White Kyurem has a higher Special Defense stat of 100 compared to Kyurem Black Kyurem's Special Defense stat of 90.\n\nAll other stats including HP, Speed, and Generation are the same for both forms. Both are also classified as Dragon/Ice types and are marked as TRUE for a specific unidentified attribute."

In [48]:
from langchain.tools import Tool

def excel_search_tool(input_text:str):
    response=query_engine.query(input_text)
    return str(response)

excel_retriever_tool=Tool(
    name="Excel_search",
    func=excel_search_tool,
    description="A tool to search for and extract information from Excel documents"
)

tools=[pdf_retriever_tool,excel_retriever_tool]
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [53]:
response=agent_executor.invoke({"input":"Name all Grass type Pokemons ?"})
print(response)



> Entering new AgentExecutor chain...
There are many Grass-type Pokémon across the various generations of Pokémon games. Here is a list of some notable Grass-type Pokémon from various generations:

1. **Bulbasaur**
2. **Ivysaur**
3. **Venusaur**
4. **Oddish**
5. **Gloom**
6. **Vileplume**
7. **Paras**
8. **Parasect**
9. **Bellsprout**
10. **Weepinbell**
11. **Victreebel**
12. **Exeggcute**
13. **Exeggutor**
14. **Tangela**
15. **Chikorita**
16. **Bayleef**
17. **Meganium**
18. **Bellossom**
19. **Hoppip**
20. **Skiploom**
21. **Jumpluff**
22. **Sunkern**
23. **Sunflora**
24. **Treecko**
25. **Grovyle**
26. **Sceptile**
27. **Shroomish**
28. **Breloom**
29. **Roselia**
30. **Cacnea**
31. **Cacturne**
32. **Lileep**
33. **Cradily**
34. **Tropius**
35. **Turtwig**
36. **Grotle**
37. **Torterra**
38. **Budew**
39. **Roserade**
40. **Carnivine**
41. **Snover**
42. **Abomasnow**
43. **Tangrowth**
44. **Leafeon**
45. **Shaymin**
46. **Snivy**
47. **Servine**
48. **Serperior**
49. **Pansage*

In [52]:
response = agent_executor.invoke({"input":"What is query,key and value in paper attention is all you need ?"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `Pdf_search` with `{'query': 'query, key, value in Attention is All You Need'}`


values.
In practice, we compute the attention function on a set of queries simultaneously, packed together
into a matrix Q. The keys and values are also packed together into matrices KandV. We compute
the matrix of outputs as:
Attention( Q, K, V ) = softmax(QKT
√dk)V (1)
The two most commonly used attention functions are additive attention [ 2], and dot-product (multi-
plicative) attention. Dot-product attention is identical to our algorithm, except for the scaling factor
of1√dk. Additive attention computes the compatibility function using a feed-forward network with
a single hidden layer. While the two are similar in theoretical complexity, dot-product attention is
much faster and more space-efficient in practice, since it can be implemented using highly optimized

Scaled Dot-Product Attention
 Multi-Head Attention
Figure 2: (left) Scaled Dot-Product Att